# Implementacija Neuronske Mreže

* Implementiraj neuronsku mrežu. 
* Mreža mora imati mehanizam za dodavanje slojeva. 
* Sloj mora imati aktivacijsku funkciju.
* Mrežu je neophodno trenirati koristeći gradijentni spust, koristeći odgovarajuću loss (cost) funkciju.
* Neuronska mreža mora imati metod `predict`.

In [258]:
import numpy as np
import random
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report

In [259]:
def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1-np.tanh(x)**2

def sigmoid(x):
    return 1.0/(1.0+np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x)*(1-sigmoid(x))

In [260]:
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2))/2

def mse_prime(y_true, y_pred):
    return (y_pred-y_true)/y_true.size

In [261]:
class FCLayer:
    def __init__(self, input_size, output_size, activation, activation_prime):
        self.activation_f = activation
        self.activation_prime_f = activation_prime
        
        self.w = np.random.rand(output_size, input_size) - 0.5
        self.b = np.random.rand(output_size,1) - 0.5
        
        self.zj = None
        
        self.input = None
        self.output = None

    
    def forward_propagation(self, input_data):
        self.input = input_data
        self.zj = np.dot(self.w, self.input) + self.b
        self.output = self.activation_f(self.zj)
        return self.activation_f(self.zj)

        

    def backward_propagation(self, output_error, learning_rate):
        delta = output_error * self.activation_prime_f(self.zj)
        w_error = np.dot(delta, self.input.T)
        
        self.w -= learning_rate * w_error
        self.b -= learning_rate * output_error
        return np.dot(self.w.T,delta)
        

In [262]:
class Network:
    
    def __init__(self):
        self.layers = []
        self.cost = None
        self.cost_prime = None
            
    def add(self, layer):
        self.layers.append(layer)
        
    def use(self, cost, cost_prime):
        self.cost = cost
        self.cost_prime = cost_prime
   

    def fit_gradient_descent(self, x_train, y_train, epochs, learning_rate):
        for epoch in range(epochs):
            i = np.random.randint(len(y_train))
            output = x_train[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            error = self.cost_prime(y_train[i], output)
            for layer in reversed(self.layers):
                error = layer.backward_propagation(error, learning_rate)
    
    
    def predict(self, input_data):
        samples = len(input_data)
        result = []
        
        for i in range(samples):
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)
            
        return result

In [263]:
# load MNIST from server
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# training data : 60000 samples
# reshape and normalize input data
x_train = x_train.reshape(60000,784,1)
y_train = to_categorical(y_train,10)
y_train = y_train.reshape(60000,10,1)

x_test = x_test.reshape(10000,784,1)
y_test = to_categorical(y_test,10)


x_train = x_train.astype('float32')
x_train /= 255

x_test = x_test.astype('float32')
x_test /= 255



Instanciraj neuronsku mrežu, sa sigmoidnom funkciojom kao aktivacijskom funkcijom.

Za loss funkciju koristi mse, istreniraj mrežu sa learning_rate = 0.1 i 600000 epoha, kreiraj predikcije i ispiši classification report...

In [264]:
# NN from first example
net = Network()

net.add(FCLayer(28*28, 15, sigmoid, sigmoid_prime))

net.add(FCLayer(15, 10, sigmoid, sigmoid_prime))

In [265]:
net.use(mse, mse_prime)
net.fit_gradient_descent(x_train, y_train, epochs=600000, learning_rate=0.1)

In [266]:
out = net.predict(x_test)

In [267]:
print(classification_report(y_test, to_categorical(np.argmax(np.reshape(np.array(out), (-1,10)), axis=1), 10)))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94       980
           1       0.94      0.96      0.95      1135
           2       0.89      0.87      0.88      1032
           3       0.91      0.85      0.88      1010
           4       0.90      0.90      0.90       982
           5       0.85      0.81      0.83       892
           6       0.91      0.93      0.92       958
           7       0.85      0.90      0.87      1028
           8       0.85      0.85      0.85       974
           9       0.89      0.87      0.88      1009

   micro avg       0.89      0.89      0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000
 samples avg       0.89      0.89      0.89     10000



Kreiraj mrežu komplikovanije strukture (više slojeva), istreniraj sa istim parametrima kao u prethodnom slučaju i uporedi rezultate.

In [268]:
# More complex network
net = Network()

net.add(FCLayer(28*28, 256, sigmoid, sigmoid_prime))

net.add(FCLayer(256, 64, sigmoid, sigmoid_prime))

net.add(FCLayer(64, 32, sigmoid, sigmoid_prime))

net.add(FCLayer(32, 10, sigmoid, sigmoid_prime))

In [269]:
net.use(mse, mse_prime)
net.fit_gradient_descent(x_train, y_train, epochs=600000, learning_rate=0.1)

In [270]:
out = net.predict(x_test)

In [271]:
print(classification_report(y_test, to_categorical(np.argmax(np.reshape(np.array(out), (-1,10)), axis=1),10)))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       980
           1       0.98      0.99      0.98      1135
           2       0.95      0.95      0.95      1032
           3       0.95      0.95      0.95      1010
           4       0.96      0.94      0.95       982
           5       0.95      0.93      0.94       892
           6       0.96      0.96      0.96       958
           7       0.95      0.95      0.95      1028
           8       0.94      0.95      0.94       974
           9       0.93      0.94      0.93      1009

   micro avg       0.95      0.95      0.95     10000
   macro avg       0.95      0.95      0.95     10000
weighted avg       0.95      0.95      0.95     10000
 samples avg       0.95      0.95      0.95     10000

